In [9]:
import os
import json
import sqlite3
from openai import OpenAI
import logging
from tqdm import tqdm
from dotenv import load_dotenv
from openai import RateLimitError
import backoff
import yaml
from LLMDataProcessing import create_db_tables, process_all_prompts, fetch_prompts_from_db, save_results_to_db

In [10]:
# Load environment variables from .env file
load_dotenv()

# Set up logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

# Initialize OpenAI client
client = OpenAI(api_key=os.getenv('OPENAI_API_KEY'))
@backoff.on_exception(backoff.expo, RateLimitError)

def main():
    # Load environment variables
    load_dotenv()

    # Set up logging
    logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

    # Initialize OpenAI client
    client = OpenAI(api_key=os.getenv('OPENAI_API_KEY'))

    # Load configuration from YAML file
    with open('config.yaml', 'r') as config_file:
        config = yaml.safe_load(config_file)

    # Database path
    db_path = config['db_path']
    
    
    # Ensure database file exists
    if not os.path.exists(db_path):
        logging.error(f"Database file not found: {db_path}")
        return

    try:
        # Connect to the database
        with sqlite3.connect(db_path) as conn:
            # Create necessary tables
            create_db_tables(conn)
            
            # Process all prompts from the newspaper_data table
            process_all_prompts(conn, client)
            
            logging.info("Processing completed successfully.")
    
    except sqlite3.Error as e:
        logging.error(f"SQLite error: {e}")
    except Exception as e:
        logging.error(f"An unexpected error occurred: {e}")
    finally:
        logging.info("Script execution completed.")

if __name__ == "__main__":
    main()

Processing prompts:   0%|          | 0/47 [00:00<?, ?it/s]2024-08-24 16:28:46,126 - INFO - Processing prompt for row_id 1: {"custom_id": "dark-77574-1-16-1", "method": "POST", "url": "/v1/chat/completions", "body": {"model": "gpt-4o-mini-2024-07-18", "messages": [{"role": "system", "content": "You are extracting information about music concerts and operas from Swedish newspaper sources and organizing it into structured JSON objects. Your inputs are OCR-ed data about performances, and events from various sources that may have small mistakes you need to correct. You know that the current newspaper being processed was written on 1908.04.26.\nTasks:\n- Extract detailed information about concerts, including names, and structuring it into JSON objects\n- Including all necessary fields including date, name, venue, organizer, type of concert\n- data formatting consistency in \"DD.MM.YY\" format\n-Noting missing data with NaN\n- Maintaining accuracy in the Swedish context for language and cultu

KeyboardInterrupt: 